In [ ]:
import jaydebeapi
from os import environ
from os import listdir
from os.path import isfile, join
import pandas as pd

hivepath = "/opt/mapr/hive/hive-2.1/lib/"
commonpath = "/opt/mapr/hadoop/hadoop-2.7.0/share/hadoop/common/lib"
hivejars = [join(hivepath, f) for f in listdir(hivepath) if isfile(join(hivepath, f)) and f.endswith(".jar")]
commonjars = [join(commonpath, f) for f in listdir(commonpath) if isfile(join(commonpath, f)) and f.endswith(".jar")]

alljars = list()
alljars.extend(hivejars)
alljars.extend(commonjars)
alljars.append('/opt/mapr/lib/hadoop-common-2.7.0.jar')
conn = jaydebeapi.connect('org.apache.hive.jdbc.HiveDriver',
'jdbc:hive2://{}:10000/default;auth=maprsasl;saslQop=auth-conf'.format(environ['HIVE_METASTORE_HOST']), ["",""], 
jars=alljars)

df1 = pd.read_sql("""select 
        state,
        count(*) as n,
        cast(avg(loan_amount) as decimal(12,2)) as loan_amount,
        cast(avg(dti) as decimal(12,2)) as dti,
        avg(cast(approved as decimal(12,2))) as avg_approved
    from (select
        loan_amount,
        fico_score_group_fnl,
        case when dti >= 10.0
            then 10.0
            else dti
        end as dti,
        emp_length,
        case when state in ('CA', 'NY', 'MN', 'IL', 'FL', 'WA', 'MA', 'TX', 'GA', 'OH', 'NJ', 'VA', 'MI')
            then state
            else 'Other'
        end as state,
        loan_title,
        approved
    from default.lendingclub2017060401
    where loan_title in('Debt Consolidation', 'Other', 'Home/Home Improvement', 'Payoff Credit Card', 'Car Payment/Loan',
    'Business Loan', 'Health/Medical', 'Moving', 'Wedding/Engagement', 'Vacation', 'College', 'Renewable Energy', 'Payoff Bills',
    'Personal Loan', 'Motorcycle')) dfFnl
    group by state
    order by avg_approved desc""", conn)
conn.close()

In [ ]:
df1